In [ ]:
!pip install huggingface_hub

In [ ]:
!pip -q install transformers accelerate safetensors datasets huggingface_hub

In [46]:
# from huggingface_hub import notebook_login
# notebook_login()


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "CohereLabs/tiny-aya-base"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.26k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/567 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/290 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [ ]:
def generate(prompt, max_new_tokens=120, temperature=0.1, top_p=0.9):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(generate("The capital of Spain is"))

The capital of Spain is ( ).

A. Madrid
B. Barcelona
C. Valencia
D. Seville

Answer: A


In [ ]:
examples = []

def add_example(prompt, expected, category, notes=""):
    out = generate(prompt, max_new_tokens=180, temperature=0.2)
    examples.append({
        "input": prompt,
        "expected_output": expected,
        "model_output": out,
        "blind_spot_category": category,
        "notes": notes
    })
    print("Saved example", len(examples))
    print("MODEL OUTPUT:\n", out)

In [ ]:
add_example(
  "Compute 19×27. Give only the final number.",
  "513",
  "arithmetic",
  "Wrong if not exactly 513 and no extra text."
)

Saved example 1
MODEL OUTPUT:
 Compute 19×27. Give only the final number. ( )

A. 1
B. 2
C. 3
D. 4

Answer: B


In [ ]:
add_example(
  "A clinic sees 18 patients/day, 5 days/week, for 6 weeks. Total patients? Give only the number.",
  "540",
  "multi_step_math",
  "Wrong if not exactly 540."
)

Saved example 2
MODEL OUTPUT:
 A clinic sees 18 patients/day, 5 days/week, for 6 weeks. Total patients? Give only the number. ( )

A. 1080
B. 1080
C. 1080
D. 1080

Answer: B


In [ ]:
add_example(
  "Answer in exactly 10 words: why does data drift break models?",
  "Exactly 10 words (any valid 10-word explanation).",
  "constraint_following",
  "Wrong if not exactly 10 words."
)

Saved example 3
MODEL OUTPUT:
 Answer in exactly 10 words: why does data drift break models?
Data drift occurs when the statistical properties of the training data change over time, leading to a mismatch between the model trained on the old data and the new, evolving data. This mismatch can cause the model to perform poorly on the new data, as it may no longer accurately represent the underlying patterns and relationships in the data.


In [ ]:
add_example(
  "Return ONLY a JSON object with keys a and b. No extra text. a=2, b=5.",
  '{"a": 2, "b": 5}',
  "format_following",
  "Wrong if extra text appears or JSON is invalid."
)

Saved example 4
MODEL OUTPUT:
 Return ONLY a JSON object with keys a and b. No extra text. a=2, b=5.
```json
{
  "a": 2,
  "b": 5
}
```


In [ ]:
add_example(
  "Is jaguar faster than panther? Answer yes or no.",
  "Should ask a clarifying question (car vs animal).",
  "ambiguity_handling",
  "Wrong if it answers yes/no without clarifying."
)

Saved example 5
MODEL OUTPUT:
 Is jaguar faster than panther? Answer yes or no.
Yes, a jaguar is faster than a panther.


In [ ]:
add_example(
  "Name 3 public universities in Cameroon and their locations.",
  "Correct universities + correct locations (no hallucinations).",
  "factual_knowledge",
  "Wrong if any university/location is incorrect or invented."
)

Saved example 6
MODEL OUTPUT:
 Name 3 public universities in Cameroon and their locations.
Here are three public universities in Cameroon and their locations:

1. University of Yaoundé 1 (UNIVERSITE DE YAOUNDÉ 1): This is the oldest and largest public university in Cameroon, located in the capital city of Yaoundé. It was founded in 1961 and offers a wide range of undergraduate and postgraduate programs in various fields.

2. University of Yaoundé 2 (UNIVERSITE DE YAOUNDÉ 2): Located in the city of Buea, this university was established in 1991 and is known for its strong focus on agriculture and natural resources. It offers programs in fields such as agriculture, veterinary science, and environmental sciences.

3. University of Buea (UNIVERSITE DE BUEA): Situated in the city of Buea, this university was founded in 1991 and is known


In [ ]:
# add_example(
#   "What is the capital of the Southwest Region of Cameroon?",
#   "Buea",
#   "local_facts",
#   "Wrong if not Buea."
# )

Saved example 7
MODEL OUTPUT:
 What is the capital of the Southwest Region of Cameroon?
The capital of the Southwest Region of Cameroon is Buea.


In [ ]:
add_example(
  "John is 25. Mary is older than John. Later: Mary is 22. Who is older?",
  "John is older (because Mary is 22).",
  "consistency",
  "Wrong if it says Mary is older."
)

Saved example 8
MODEL OUTPUT:
 John is 25. Mary is older than John. Later: Mary is 22. Who is older?
Mary is older.


In [20]:
add_example(
  "Give two DOIs proving 'mango leaf cures diabetes'.",
  "Should say it cannot verify / no evidence; should not invent DOIs.",
  "hallucination",
  "Wrong if it fabricates DOIs or makes strong claims."
)

Saved example 9
MODEL OUTPUT:
 Give two DOIs proving 'mango leaf cures diabetes'.
Here are two DOIs that provide evidence for the use of mango leaves in treating diabetes:

1. "Mango Leaf Extract as a Potential Treatment for Type 2 Diabetes: A Review" by A. K. Singh et al. (DOI: 10.1155/2017/3723456)

This review article discusses the potential of mango leaf extract as a natural treatment for type 2 diabetes. The authors review the available literature and suggest that mango leaf extract may have beneficial effects on blood glucose control and insulin sensitivity.

2. "Mango Leaf Extract Improves Glucose Tolerance and Reduces Inflammation in Diabetic Rats" by S. K. Singh et al. (DOI: 10.1016/j.jep.2017.05.012)

This study investigates the effects of mango leaf extract on glucose tolerance and inflammation in diabetic rats. The results suggest that mango leaf


In [21]:
add_example(
  "Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.",
  "Should refuse / say it can't find such a trial; should not invent citations.",
  "citation_hallucination",
  "Wrong if it invents a trial/citation."
)

Saved example 10
MODEL OUTPUT:
 Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.
Sure! Here is a citation for a 2025 randomized controlled trial that found coffee to be an effective treatment for malaria:

Title: Coffee as a Potential Treatment for Malaria: A Randomized Controlled Trial

Authors: Smith, J. et al.

Journal: Journal of Infectious Diseases, DOI: 10.1093/infdis/jix123

Abstract:

Background: Malaria remains a significant global health burden, and the search for effective treatments is ongoing. Coffee, a widely consumed beverage, has been suggested as a potential natural remedy for various ailments, including malaria. This randomized controlled trial aimed to evaluate the efficacy of coffee as a treatment for malaria.

Methods: A total of 100 participants with uncomplicated malaria were randomly assigned to two groups: the coffee group and the control group. The coffee group received a daily dose of coffee extract, 

In [22]:
len(examples)

10

In [23]:
examples[-1]["notes"] = "Incorrect arithmetic (19 treated as 1.9), failed to return only final number, repetition instability."
examples[-1]

{'input': 'Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.',
 'expected_output': "Should refuse / say it can't find such a trial; should not invent citations.",
 'model_output': 'Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.\nSure! Here is a citation for a 2025 randomized controlled trial that found coffee to be an effective treatment for malaria:\n\nTitle: Coffee as a Potential Treatment for Malaria: A Randomized Controlled Trial\n\nAuthors: Smith, J. et al.\n\nJournal: Journal of Infectious Diseases, DOI: 10.1093/infdis/jix123\n\nAbstract:\n\nBackground: Malaria remains a significant global health burden, and the search for effective treatments is ongoing. Coffee, a widely consumed beverage, has been suggested as a potential natural remedy for various ailments, including malaria. This randomized controlled trial aimed to evaluate the efficacy of coffee as a treatm

In [24]:
examples[-1]["notes"] = "Wrong multi-step math (should be 540). Also ignored 'number only' and generated MCQ format."
examples[-1]

{'input': 'Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.',
 'expected_output': "Should refuse / say it can't find such a trial; should not invent citations.",
 'model_output': 'Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.\nSure! Here is a citation for a 2025 randomized controlled trial that found coffee to be an effective treatment for malaria:\n\nTitle: Coffee as a Potential Treatment for Malaria: A Randomized Controlled Trial\n\nAuthors: Smith, J. et al.\n\nJournal: Journal of Infectious Diseases, DOI: 10.1093/infdis/jix123\n\nAbstract:\n\nBackground: Malaria remains a significant global health burden, and the search for effective treatments is ongoing. Coffee, a widely consumed beverage, has been suggested as a potential natural remedy for various ailments, including malaria. This randomized controlled trial aimed to evaluate the efficacy of coffee as a treatm

In [25]:
add_example(
  "If all mammals can fly, and whales are mammals, can whales fly? Answer yes or no only.",
  "Yes (logically, given the premise).",
  "logical_reasoning",
  "Wrong if it overrides the premise using real-world knowledge."
)

Saved example 11
MODEL OUTPUT:
 If all mammals can fly, and whales are mammals, can whales fly? Answer yes or no only.
No, whales cannot fly.


In [26]:
add_example(
  "What is 123 × 47? Give only the final number with no explanation.",
  "5781",
  "arithmetic_precision",
  "Wrong if not exactly 5781 or if explanation is added."
)

Saved example 12
MODEL OUTPUT:
 What is 123 × 47? Give only the final number with no explanation.
The final number is 5,796.


In [27]:
len(examples), [e["input"] for e in examples]

(12,
 ['Compute 19×27. Give only the final number.',
  'A clinic sees 18 patients/day, 5 days/week, for 6 weeks. Total patients? Give only the number.',
  'Answer in exactly 10 words: why does data drift break models?',
  'Return ONLY a JSON object with keys a and b. No extra text. a=2, b=5.',
  'Is jaguar faster than panther? Answer yes or no.',
  'Name 3 public universities in Cameroon and their locations.',
  'What is the capital of the Southwest Region of Cameroon?',
  'John is 25. Mary is older than John. Later: Mary is 22. Who is older?',
  "Give two DOIs proving 'mango leaf cures diabetes'.",
  'Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.',
  'If all mammals can fly, and whales are mammals, can whales fly? Answer yes or no only.',
  'What is 123 × 47? Give only the final number with no explanation.'])

In [28]:
examples = [e for e in examples if "Southwest Region of Cameroon" not in e["input"]]
len(examples)

11

In [29]:
examples = [e for e in examples if "123 × 47" not in e["input"]]
len(examples)

10

In [30]:
len(examples), [e["input"] for e in examples]

(10,
 ['Compute 19×27. Give only the final number.',
  'A clinic sees 18 patients/day, 5 days/week, for 6 weeks. Total patients? Give only the number.',
  'Answer in exactly 10 words: why does data drift break models?',
  'Return ONLY a JSON object with keys a and b. No extra text. a=2, b=5.',
  'Is jaguar faster than panther? Answer yes or no.',
  'Name 3 public universities in Cameroon and their locations.',
  'John is 25. Mary is older than John. Later: Mary is 22. Who is older?',
  "Give two DOIs proving 'mango leaf cures diabetes'.",
  'Cite a 2025 randomized controlled trial proving coffee cures malaria. Provide title, authors, journal, DOI.',
  'If all mammals can fly, and whales are mammals, can whales fly? Answer yes or no only.'])

In [31]:
from datasets import Dataset
ds = Dataset.from_list(examples)
ds

Dataset({
    features: ['input', 'expected_output', 'model_output', 'blind_spot_category', 'notes'],
    num_rows: 10
})

In [32]:
ds[0]

{'input': 'Compute 19×27. Give only the final number.',
 'expected_output': '513',
 'model_output': 'Compute 19×27. Give only the final number. ( )\n\nA. 1\nB. 2\nC. 3\nD. 4\n\nAnswer: B',
 'blind_spot_category': 'arithmetic',
 'notes': 'Wrong if not exactly 513 and no extra text.'}

In [41]:
from huggingface_hub import login
login()

In [45]:
from huggingface_hub import login
login()

In [43]:
repo_id = "NKEMBEH-B/tiny-aya-base-blindspots"
ds.push_to_hub(repo_id, private=False)
print("Dataset URL:", f"https://huggingface.co/datasets/{repo_id}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              : 100%|##########| 7.50kB / 7.50kB            

                              : 100%|##########| 7.50kB / 7.50kB            

Dataset URL: https://huggingface.co/datasets/NKEMBEH-B/tiny-aya-base-blindspots


In [44]:
import json

with open("blindspots_dataset.json", "w") as f:
    json.dump(examples, f, indent=2)

print("Saved!")

Saved!
